In [2]:
import cv2
import tensorflow as tf
import numpy as np


2024-05-15 13:10:36.294955: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 13:10:37.906124: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import tensorflow as tf
import numpy as np


# Especifica la ruta al archivo de configuración del modelo en formato JSON
config_path = '/home/ezequielgarcia/Escritorio/proyecto reconocimiento/modelo_ultimo_este_si/modelo_entrenado/config.json'

# Carga la configuración del modelo
with open(config_path, 'r') as f:
    config = f.read()

# Crea el modelo a partir de la configuración
model = tf.keras.models.model_from_json(config)

# Especifica la ruta al archivo .h5 que contiene los pesos del modelo
weights_path = '/home/ezequielgarcia/Escritorio/proyecto reconocimiento/modelo_ultimo_este_si/modelo_entrenado/model.weights.h5'

# Carga los pesos del modelo
model.load_weights(weights_path)




/home/ezequielgarcia/Entorno V/zeke/lib64/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 4 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
#Categorizar una imagen de internet
from PIL import Image
import requests
from io import BytesIO
import cv2

def categorizar(url):
  respuesta = requests.get(url)
  img = Image.open(BytesIO(respuesta.content))
  img = np.array(img).astype(float)/255

  img = cv2.resize(img, (224,224))
  prediccion = model.predict(img.reshape(-1, 224, 224, 3))
  return prediccion

In [5]:
#0 = gato, 1 = perro
url = "https://i.ebayimg.com/images/g/q10AAOSwuAxleGFu/s-l1600.jpg"
prediccion = categorizar(url)
print(prediccion)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.00269047 0.99730957]]


In [6]:
def preprocess_image(image):
    # Redimensionar la imagen a las dimensiones de entrada del modelo
    resized_image = cv2.resize(image, (224, 224))  # Cambia las dimensiones a (224, 224) para que coincidan con el modelo
    # Normalizar los valores de píxeles (si es necesario)
    normalized_image = resized_image / 255.0  # Ajusta esto según cómo hayas preprocesado tus datos de entrenamiento
    # Expandir las dimensiones de la imagen para que coincidan con el formato de entrada del modelo
    processed_image = np.expand_dims(normalized_image, axis=0)
    return processed_image


In [ ]:
import cv2
import numpy as np
import time

# Función para preprocesar la imagen capturada
def preprocess_image(image):
    resized_image = cv2.resize(image, (224, 224))
    normalized_image = resized_image / 255.0
    processed_image = np.expand_dims(normalized_image, axis=0)
    return processed_image

# Configuración de la cámara
cap = cv2.VideoCapture(0)

# Define las coordenadas (x, y) donde quieres colocar el texto
x = 50
y = 50

while True:
    ret, frame = cap.read()

    if not ret:
        print("No se pudo capturar un frame de la cámara.")
        break

    # Procesar el frame actual
    processed_frame = preprocess_image(frame)

    # Clasificar la imagen utilizando el modelo y almacenar las probabilidades
    prediction = model.predict(processed_frame)
    probability_gato = prediction[0][0]
    probability_perro = prediction[0][1]

    # Dibujar un recuadro si se detecta un gato
    if probability_gato >= 0.85:
        # Dibujar un rectángulo alrededor de la región de interés
        cv2.putText(frame, "Gato", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    # Dibujar un recuadro si se detecta un perro
    elif probability_perro >= 0.85:
        # Dibujar un rectángulo alrededor de la región de interés
        cv2.putText(frame, "Perro", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Mostrar el frame con la información procesada
    cv2.imshow('Procesamiento', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

